In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 500

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

In [7]:
# pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [8]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )



In [9]:
len(pt_features[pt_features['isCase']==True])

15209

In [10]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,data_start,matchid,reason_for_removal,age_at_index_date,insomnia,stroke,non_stroke_vascular_disease,hypertension,diabetes,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,chronic_pulmonary_disease,epilepsy,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,other_sedatives_100_pdds,antipsychotics_100_pdds,depot_antipsychotics_100_pdds,antidepressants_100_pdds,non_insomnia_GP_consultations,benzo_and_z_drugs_any,insomnia_any,insomnia_count:0,insomnia_count:1_5,insomnia_count:above_5,non_insomnia_GP_consultations:0,non_insomnia_GP_consultations:1_10,non_insomnia_GP_consultations:11_100,non_insomnia_GP_consultations:101_1000,non_insomnia_GP_consultations:above_1000,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,antidepressant_pdds:00000,antidepressant_pdds:00001_10,antidepressant_pdds:00011_100,antidepressant_pdds:00101_1000,antidepressant_pdds:01001_10000,antidepressant_pdds:10000_and_above,antidepressant_pdds:10000_and_above,antipsychotic_pdds:00000,antipsychotic_pdds:00001_10,antipsychotic_pdds:00011_100,antipsychotic_pdds:00101_1000,antipsychotic_pdds:01001_10000,antipsychotic_pdds:10000_and_above,antipsychotic_pdds:10000_and_above,depot_antipsychotic_pdds:00000,depot_antipsychotic_pdds:00001_10,depot_antipsychotic_pdds:00011_100,depot_antipsychotic_pdds:00101_1000,depot_antipsychotic_pdds:01001_10000,depot_antipsychotic_pdds:10000_and_above,depot_antipsychotic_pdds:10000_and_above,other_sedative_pdds:00000,other_sedative_pdds:00001_10,other_sedative_pdds:00011_100,other_sedative_pdds:00101_1000,other_sedative_pdds:01001_10000,other_sedative_pdds:10000_and_above,other_sedative_pdds:10000_and_above,benzo_and_z_drug_pdds:00000,benzo_and_z_drug_pdds:00001_10,benzo_and_z_drug_pdds:00011_100,benzo_and_z_drug_pdds:00101_1000,benzo_and_z_drug_pdds:01001_10000,benzo_and_z_drug_pdds:10000_and_above,benzo_and_z_drug_pdds:10000_and_above,mood_stabiliser_pdds:00000,mood_stabiliser_pdds:00001_10,mood_stabiliser_pdds:00011_100,mood_stabiliser_pdds:00101_1000,mood_stabiliser_pdds:01001_10000,mood_stabiliser_pdds:10000_and_above,mood_stabiliser_pdds:10000_and_above
24894,10172098,29,98,1,2009-10-06,False,NaN,2013-05-03,1996-05-21,33474,NaN,80,0,0,0,0,0,1,0,0,1,0,0.0,0.0,0.000000,0.000000,0.0,0.0,43,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0
21005,8108391,14,391,1,2011-04-16,True,1350.0,2013-04-19,1997-05-27,57950,NaN,97,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.0,84,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0
14742,6933071,23,71,1,2005-04-21,False,NaN,2007-07-31,1993-01-31,57369,NaN,82,0,0,1,0,0,0,0,0,1,0,0.0,0.0,0.068692,0.232046,0.0,0.0,106,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0
14745,529084,14,84,1,2005-03-08,True,1917.0,2006-07-31,1992-01-31,11031,NaN,91,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.000000,0.000000,0.0,0.0,83,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0
17402,4756206,30,206,0,2008-06-23,False,NaN,2013-03-08,1998-01-31,35113,NaN,78,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.000000,0.116023,0.0,0.0,129,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0

In [11]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'data_start', 'matchid',
       'reason_for_removal', 'age_at_index_date', 'insomnia', 'stroke',
       'non_stroke_vascular_disease', 'hypertension', 'diabetes',
       'mental_illness_non_smi', 'mental_illness_smi', 'sleep_apnoea',
       'chronic_pulmonary_disease', 'epilepsy', 'mood_stabilisers_100_pdds',
       'benzo_and_z_drugs_100_pdds', 'other_sedatives_100_pdds',
       'antipsychotics_100_pdds', 'depot_antipsychotics_100_pdds',
       'antidepressants_100_pdds', 'non_insomnia_GP_consultations',
       'benzo_and_z_drugs_any', 'insomnia_any', 'insomnia_count:0',
       'insomnia_count:1_5', 'insomnia_count:above_5',
       'non_insomnia_GP_consultations:0', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000', 'age_at_index_date:65-69',


In [12]:
characteristics = [column for column in pt_features.columns if column not in [
    'patid',  'yob', 'pracid', 'index_date', 'isCase',
    'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [13]:
baselines = add_baseline_characteristics(characteristics,pt_features)

In [14]:
len(pt_features[pt_features['isCase']==False]),len(pt_features[pt_features['isCase']==True])

(15209, 15209)

In [15]:
baselines

,Cases,Controls,P value
female,10055 (66.1%),10055 (66.1%),1.000
stroke,1662 (10.9%),1120 (7.4%),0.000
non_stroke_vascular_disease,4513 (29.7%),3465 (22.8%),0.000
hypertension,7181 (47.2%),6642 (43.7%),0.000
diabetes,1745 (11.5%),1259 (8.3%),0.000
mental_illness_non_smi,5622 (37.0%),3556 (23.4%),0.000
mental_illness_smi,898 (5.9%),528 (3.5%),0.000
sleep_apnoea,25 (0.2%),19 (0.1%),0.365
chronic_pulmonary_disease,3449 (22.7%),2512 (16.5%),0.000
epilepsy,288 (1.9%),165 (1.1%),0.000


In [17]:
baselines = add_baseline_characteristics(characteristics,pt_features)

In [19]:
baselines

,Cases,Controls,P value
female,10055 (100.0%),10055 (100.0%),-
stroke,983 (9.8%),690 (6.9%),0.000
non_stroke_vascular_disease,2715 (27.0%),2036 (20.2%),0.000
hypertension,5016 (49.9%),4627 (46.0%),0.000
diabetes,991 (9.9%),724 (7.2%),0.000
mental_illness_non_smi,4036 (40.1%),2560 (25.5%),0.000
mental_illness_smi,670 (6.7%),401 (4.0%),0.000
sleep_apnoea,6 (0.1%),4 (0.0%),0.527
chronic_pulmonary_disease,2334 (23.2%),1675 (16.7%),0.000
epilepsy,173 (1.7%),95 (0.9%),0.000
